In [1]:
import pandas as pd
import re

In [14]:
## pulling from pickle file created in Step 1

In [2]:
df = pd.read_pickle('/Users/onb/Documents/Analysis/DoubleVerify Analysis/Data/Safety Flag/DV_Safety_Flag_Risk_Data.pkl')

In [3]:
df.head()

,CHANNEL_ID,VIDEO_ID,SUITABILITY_CATEGORY,TAXONOMY_NAME,SCORE,DECISION,DV_RISK,vector1,vector2,simscore
3,UCMAL34ECveufC_sxmnhN5_w,BCET78lZufI,Vehicle Disasters,Seasons - Springtime,0.042186,None,Medium Risk,"{'Vehicle': 1, 'Disasters': 1}","{'Seasons': 1, 'Springtime': 1}",0.0
27,UCSnNUFvvfLRmVVytvF0EYDQ,x5VeJoDDE4Y,Adult & Sexual,Football - V1,0.012785,False,Medium Risk,"{'Adult': 1, 'Sexual': 1}","{'Football': 1, 'V1': 1}",0.0
33,UCCzHAODYr8mbnmupfhbn8sQ,lhtrk0_Mny4,Natural Disasters,Brand Safety - Weapons - V1,0.040382,None,Medium Risk,"{'Natural': 1, 'Disasters': 1}","{'Brand': 1, 'Safety': 1, 'Weapons': 1, 'V1': 1}",0.0
43,UCb-ikMqe9iS8GjG7WgGwCrQ,SXpnwi6ZfoA,Death & Injury,Drama + TV/Film - V1,0.059013,None,Medium Risk,"{'Death': 1, 'Injury': 1}","{'Drama': 1, 'TV': 1, 'Film': 1, 'V1': 1}",0.0
49,UC31Ux0TEaLel-q8-xgx0O_A,9b43mJXu3xE,Adult & Sexual,Valentine's Day - V1,0.148346,None,Low Risk,"{'Adult': 1, 'Sexual': 1}","{'Valentine': 1, 's': 1, 'Day': 1, 'V1': 1}",0.0


In [4]:
df.drop(['vector1','vector2'], axis = 1, inplace = True)

In [5]:
concepts = []

for txt in df['TAXONOMY_NAME']:
    if '4A' in txt:
        result =  re.search('- (.*) -', txt)
        concepts.append(result.group(1))
    else:
        concepts.append(txt)
        
df['CONCEPTS'] = concepts

In [6]:
ZEFR_RISK = []

for txt in df['TAXONOMY_NAME']:
    if 'Minimal' in txt:
        ZEFR_RISK.append('Minimal')
    elif 'Low' in txt:
        ZEFR_RISK.append('Low')
    elif 'Medium' in txt:
        ZEFR_RISK.append('Medium')
    elif 'High' in txt:
        ZEFR_RISK.append('High')
    else:
        ZEFR_RISK.append('No GARM')
        
df['ZEFR_RISK'] = ZEFR_RISK

In [7]:
same_risk = []

for i,v in enumerate(df['TAXONOMY_NAME']):
    if 'Minimal' in df['DV_RISK'].iloc[i] and 'Minimal' in v:
        same_risk.append(1)
    elif 'Low' in df['DV_RISK'].iloc[i] and 'Low' in v:
        same_risk.append(1)  
    elif 'Medium' in df['DV_RISK'].iloc[i] and 'Medium' in v:
        same_risk.append(1) 
    elif 'High' in df['DV_RISK'].iloc[i] and 'High' in v or 'Floor' in v:
        same_risk.append(1) 
    else:
        same_risk.append(0)

In [8]:
df['SAME_RISK'] = same_risk

In [10]:
df['DV_RISK'].unique()

array(['Medium Risk', 'Low Risk', 'High Risk'], dtype=object)

In [11]:
df['SUITABILITY_CATEGORY'].unique()

array(['Vehicle Disasters', 'Adult & Sexual', 'Natural Disasters',
       'Death & Injury', 'Profanity', 'Hate Speech & Cyberbullying',
       'Human-made Disasters', 'Crime', 'Alcohol', 'Aviation Disasters',
       'Violence', 'Terrorism', 'Drug Abuse', 'Tobacco & eCigarettes'],
      dtype=object)

In [37]:
df.to_clipboard()

In [9]:
df.head()

,CHANNEL_ID,VIDEO_ID,SUITABILITY_CATEGORY,TAXONOMY_NAME,SCORE,DECISION,DV_RISK,simscore,CONCEPTS,ZEFR_RISK,SAME_RISK
3,UCMAL34ECveufC_sxmnhN5_w,BCET78lZufI,Vehicle Disasters,Seasons - Springtime,0.042186,None,Medium Risk,0.0,Seasons - Springtime,No GARM,0
27,UCSnNUFvvfLRmVVytvF0EYDQ,x5VeJoDDE4Y,Adult & Sexual,Football - V1,0.012785,False,Medium Risk,0.0,Football - V1,No GARM,0
33,UCCzHAODYr8mbnmupfhbn8sQ,lhtrk0_Mny4,Natural Disasters,Brand Safety - Weapons - V1,0.040382,None,Medium Risk,0.0,Brand Safety - Weapons - V1,No GARM,0
43,UCb-ikMqe9iS8GjG7WgGwCrQ,SXpnwi6ZfoA,Death & Injury,Drama + TV/Film - V1,0.059013,None,Medium Risk,0.0,Drama + TV/Film - V1,No GARM,0
49,UC31Ux0TEaLel-q8-xgx0O_A,9b43mJXu3xE,Adult & Sexual,Valentine's Day - V1,0.148346,None,Low Risk,0.0,Valentine's Day - V1,No GARM,0


In [13]:
## below script used for importing above df to snowflake; I ended up just using the pickle files

In [39]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer

In [40]:
table_name = 'ZEFR_SAFETY_FLAG_RISK'
schema = 'ANALYST'

In [41]:
df.rename(columns={"simscore": "SIMSCORE"}, inplace = True)

In [42]:
def connection(schema):
    conn = snowflake.connector.connect(
        host="localhost",
        port="10015",
        user="sdm",
        password="sdm",
        account="dummy",
        protocol="http",
        warehouse="ENGINEER_WH",
        database="PROD",
        schema=schema,
    )
    return conn

In [43]:
def write_snowflake(schema,df,table_name):
    conn = connection(schema)
    success, nchunks, nrows, _ = write_pandas(conn, df, table_name, chunk_size = 1000)
    
    return success, nchunks, nrows, _

In [44]:
success, nchunks, nrows, _ = write_snowflake(schema, df, table_name)

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 199
invalid identifier 'CONCEPTS'